In [1]:
import openseespy.opensees as ops
import numpy as np
import matplotlib.pyplot as plt
import opsvis as opsv
import model as md
from builder import opsbuild as bd
from Unit import Unit
import handcalcs.render
import forallpeople as si
%matplotlib inline

In [2]:
si.environment("structural",top_level=False)

In [3]:
numBay, numFloor, firstFloor, bayWidht, storyHeight = 3,3,3,5,3
pf1 = md.createmodel(numBay, numFloor, firstFloor, bayWidht, storyHeight)

In [4]:
%%render long
#params
fc=20*Unit.MPa# Nominal concrete compressive strength
HCol  = 30*Unit.cm
BCol  = 30*Unit.cm
HBeam = 40*Unit.cm
BBeam = 30*Unit.cm

<IPython.core.display.Latex object>

In [5]:
%%render long
#params
concdensity = 24.99 * Unit.kN/Unit.m**3   # Conc density
wBeam = ((20 +  0.3*10)*Unit.kN/Unit.m)   # dist. load on beam

<IPython.core.display.Latex object>

In [7]:
%%render long
Ec = 5000*Unit.MPa*(fc/Unit.MPa)**0.5     # Young's modulus
nu = 0.2                        # Poisson's ratio
Gc = Ec/(2*(1+nu))              # Shear modulus
# unconfined concrete
Kres = 0.2          # ratio of residual/ultimate to maximum stress
fc1U = -fc          # UNCONFINED concrete (todeschini parabolic model), maximum stress
eps1U = -0.003      # strain at maximum strength of unconfined concrete
fc2U = Kres*fc1U    # ultimate stress
eps2U = -0.01       # strain at ultimate stress
# confined concrete
Kfc = 1.2           # ratio of confined to unconfined concrete strength
fc1C = Kfc*fc1U     # CONFINED concrete (mander model), maximum stress
eps1C = 2*fc1C/Ec   # strain at maximum stres
fc2C = Kres*fc1C    # ultimate stress
eps2C = 5*eps1C     # strain at ultimate stress 
# tensile-strength properties
Lambda = 0.1        # ratio between unloading slope at eps2 and initial slope Ec
fct = 0.14*fc       # tensile strength +tension
Ets = fct/0.002     # tension softening stiffness

# REINFORCING STEEL
#============================================================================================================
fsy = 500*Unit.MPa;     # Yield stress
Es = 2*10**5*Unit.MPa;     # Young's modulus
bs = 0.01           # strain-hardening ratio
R0 = 18             # control the transition from elastic to plastic branches
cR1 = 0.925         # control the transition from elastic to plastic branches
cR2 = 0.15          # control the transition from elastic to plastic branches
minStrain = -0.1    # minimum steel strain in the fibers (steel buckling)
maxStrain = 0.1     # maximum steel strain in the fibers (steel rupture)

# Elastic section properties
#==================================================================================================================
kCol=0.8
kBeam=0.5
kv=5/6
ACol = HCol*HBeam
Avcol = kv*ACol

Acol = HCol*BCol                  # Cross-sectional area of columns
Icol = kCol*1/12*BCol*HCol**3     # Effective moment of inertia of columns
Avcol = kv*Acol                   # Shear area for beams (Timoshenko Elements)
Abeam = BBeam*HBeam               # Cross-sectional area of beams
Ibeam = kBeam*1/12*BBeam*HBeam**3 # Gross moment of inertia of beams
Avbeam = kv*Abeam                 # Shear area for beams (Timoshenko Elements)

bardiameterbeam = 0.016 #mm
bardiametercol = 0.022 #mm
bardiametercol2 = 0.016 #mm
cover = 0.05
barareabeam = 3.14*bardiameterbeam**2/4
barareacol = 3.14*bardiametercol**2/4
barareacol2 = 3.14*bardiametercol2**2/4
numBarTop=5
numBarBot = 3
numBarInt = 0

<IPython.core.display.Latex object>

In [ ]:
bd.modelbuild()

In [ ]:
nodal_mass = bd.calcnodalmass(pf1.node_dict,pf1.column_dict,pf1.beam_dict,pf1.ColumnLength_dict,pf1.BeamLength_dict,
                              Acol,Abeam,distLoadbeam=0,concDensity=concdensity)
bd.createnodes(pf1.node_dict,nodal_mass,mass=True) # Create nodes and nodal mass for modal analysis

In [ ]:
for i in pf1.floorNodes[0]:
    print(i)
    ops.fix(i, 1, 1, 1)

In [ ]:
ColTransfTag = 1
BeamTransfTag = 2
ops.geomTransf('Linear', BeamTransfTag) #Beam Tranformation Tag
ops.geomTransf('PDelta', ColTransfTag)  #Column Tranformation Tag

In [ ]:
IDconcCover = 1 # Tag for unconfined concrete material
IDconcCore = 2 # Tag for confined concrete material
IDSteel = 3 # Tag for steel material without min-max properties
IDMinMaxSteel = 4 # Tag for steel material with min-max properties
IDshearCol = 5 # Shear material tag for columns

In [ ]:
ops.uniaxialMaterial('Concrete02',IDconcCover, fc1U, eps1U, fc2U, eps2U, Lambda, fct, Ets)
ops.uniaxialMaterial('Concrete02',IDconcCore, fc1C, eps1C, fc2C, eps2C, Lambda, fct, Ets)
#ops.uniaxialMaterial('Concrete01',IDconcCover, fc1U, eps1U, fc2U, eps2U,)
#ops.uniaxialMaterial('Concrete01',IDconcCore, fc1C, eps1C, fc2C, eps2C )
ops.uniaxialMaterial('Steel02',IDSteel, fsy, Es, bs,  R0, cR1, cR2)
ops.uniaxialMaterial('MinMax', IDMinMaxSteel, IDSteel, '-min', minStrain, '-max', maxStrain)

In [ ]:
# Define sections
# ------------------------------------------------------------------------
# rectangular fiber section with one layer of steel evenly distributed around the perimeter and a confined core.
ColSecTag_Fiber = 1 # Tag for fiber column sections
ColSecTag_Fiber2 = 2
ColSecTag_Elastic = 3       # Tag for fiber column section aggregated with elastic shear section
ColSecTag_Elastic2 = 4
BeamSecTag_Fiber = 5      # Tag for fiber beam section which is located at the beam ends
ElBeamSec_Elastic = 6       # Tag for elastic sbeam section which is located at the beam interior

In [ ]:
col1_fiber_sec = bd.BuildRCrectSection(ColSecTag_Fiber, HCol, BCol, cover, cover, IDconcCore, IDconcCover, IDSteel,
                        numBarTop,barareacol ,numBarBot, barareacol,numBarInt, barareacol, 
                        nfCoreY=20, nfCoreZ=20, nfCoverY=20, nfCoverZ=20, pflag=1)
bd.plot_fiber_section(ColSecTag_Fiber,col1_fiber_sec)

In [ ]:
col2_fiber_sec = bd.BuildRCrectSection(ColSecTag_Fiber2, HCol, BCol, cover, cover, IDconcCore, IDconcCover, IDSteel,
                        numBarTop,barareacol2 ,numBarBot, barareacol2,numBarInt, barareacol2, 
                        nfCoreY=20, nfCoreZ=20, nfCoverY=20, nfCoverZ=20, pflag=1)
bd.plot_fiber_section(ColSecTag_Fiber2,col2_fiber_sec)

In [ ]:
beamfiber_sec =bd.BuildRCrectSection(BeamSecTag_Fiber, HBeam, BBeam, cover, cover, IDconcCore, IDconcCover, IDSteel,
                        numBarTop,barareabeam ,numBarBot, barareabeam,numBarInt, barareabeam,  
                        nfCoreY=20, nfCoreZ=20, nfCoverY=20, nfCoverZ=20, pflag=1)
bd.plot_fiber_section(BeamSecTag_Fiber,beamfiber_sec)

In [ ]:
# section('Elastic', secTag, E_mod, A, Iz, G_mod=None, alphaY=None)
ops.section('Elastic', ColSecTag_Elastic, Ec, Acol, Icol)
#ops.section('Aggregator', ColSecTag_Elastic, IDshearCol, 'Vy', '-section', ColSecTag_Fiber)
#ops.section('Aggregator', ColSecTag_Elastic2, IDshearCol, 'Vy', '-section', ColSecTag_Fiber2)
ops.section('Elastic', ElBeamSec_Elastic, Ec, Abeam, Ibeam)

In [ ]:
#Column integration points definitions
#Create elements
## Create Integration point 
colIntgrTag = 1
colIntgrTag2 = 2
beamIntgrTag = 3
Lpl = 0.5* HBeam
Lpl_col = 0.5* HCol
np=5
#beamIntegration('HingeRadauTwo', tag, secI, lpI, secJ, lpJ, secE)
#ops.beamIntegration('Lobatto',colIntgrTag,ColSecTag_Fiber,np)
#ops.beamIntegration('Lobatto',colIntgrTag2,ColSecTag_Fiber2,np)
ops.beamIntegration('HingeMidpoint',colIntgrTag,ColSecTag_Fiber,Lpl_col,ColSecTag_Fiber,Lpl_col,ColSecTag_Elastic)
ops.beamIntegration('HingeMidpoint',colIntgrTag2,ColSecTag_Fiber2,Lpl_col,ColSecTag_Fiber2,Lpl_col,ColSecTag_Elastic)

#Beam integration point definition
ops.beamIntegration('HingeRadau',beamIntgrTag,BeamSecTag_Fiber,Lpl,BeamSecTag_Fiber,Lpl,ElBeamSec_Elastic)

##Create Beams and Columns
for colId in pf1.ColumnLength_dict.keys():
    if 4<=colId<=9:
#           element('nonlinearBeamColumn', eleTag,          iNode           ,         jNode            , numIntgrPts,     secTag    , transfTag  , '-iter', maxIter=10, tol=1e-12, '-mass', mass=0.0, '-integration', intType)
        #ops.element('nonlinearBeamColumn', colId , pf1.column_dict[colId][0], pf1.column_dict[colId][1], np         ,ColSecTag_Fiber,ColTransfTag,'-mass',PCol)
        #   element( 'forceBeamColumn'   ,eleTag ,            iNode         ,            jNode         , transfTag  ,entegrasyonTag, '-iter' , maxIter=10 , tol=1e-12 , '-mass' , mass=0.0 )
        ops.element('forceBeamColumn'    , colId , pf1.column_dict[colId][0], pf1.column_dict[colId][1],ColTransfTag,  colIntgrTag )
    else:
        #ops.element('nonlinearBeamColumn', colId , pf1.column_dict[colId][0], pf1.column_dict[colId][1], np         ,ColSecTag_Fiber2,ColTransfTag,'-mass',PCol)
        ops.element('forceBeamColumn', colId,pf1.column_dict[colId][0], pf1.column_dict[colId][1],ColTransfTag,colIntgrTag2)
    
for beamId in pf1.beam_dict.keys():
    ops.element('forceBeamColumn',beamId, pf1.beam_dict[beamId][0], pf1.beam_dict[beamId][1],BeamTransfTag,beamIntgrTag,'-mass', wBeam/Unit.g)

In [ ]:
# Create a Plain load pattern with a Linear TimeSeries
# ------------------------------------------------------------------------
ops.timeSeries('Linear', 1)
ops.pattern('Plain', 1, 1)
for idBeam in pf1.beam_dict.keys():
    ops.eleLoad('-ele', idBeam, '-type', '-beamUniform', -wBeam, 0)

In [ ]:
T, Mratios, Mfactors, Mtots = bd.modal_analys2(3)

In [ ]:
eleForces_grav, nodalDisps_grav, BaseReactions_grav,eleSection_grav = bd.do_gravity()

In [ ]:
bd.plot_model()

## Fiber Section Nonlineer Static Pushover Analysis

# Apply loads
# ------------------------------------------------------------------------
# Triangular force distribution, (set sum = 1 so that loadfactor = Total Base Shear)
ops.timeSeries('Linear', 2)  # Define the timeSeries for the load pattern
ops.pattern('Plain', 2, 2)   # Define load pattern -- generalized
for node in pf1.node_dict.keys():
    if node in pf1.floorNodes[0]:
        continue
    if node in pf1.floorNodes[1]:
        ops.load(node,1,0,0)
    if node in pf1.floorNodes[2]:
        ops.load(node,2,0,0)

ctrlNode = max(pf1.node_dict.keys())
ctrlDOF =1
nSteps = 200
dmax= 0.1*(firstFloor+(numFloor-1)*storyHeight)
print(dmax)
LoadFactor, DispCtrlNode,sectionStrainStress,nodalDisps,eleCurvature  = bd.do_nspa(dmax, ctrlNode, ctrlDOF, nSteps,pFlag=0)
# Plot the capacity curve
# ------------------------------------------------------------------------
plt.figure()
plt.plot(DispCtrlNode,LoadFactor)
plt.xlabel('Top displacement [m]')
plt.ylabel('Base Shear Foce [kN]')
plt.grid(True)
ops.wipeAnalysis()

In [ ]:
bd.rayleigh(numMode=2,xDamp = 0.02,betaKcurr = 0.0,betaKinit = 0.0)

In [ ]:
import os
import ReadRecord as rmd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

eventname="Mammoth_Lakes"
eventlist = [i for i in os.listdir(f"./{eventname}") if i.endswith('.AT2')] #verilen deprem kayıtlarının bulunduğu klasördeki .AT2 uzantılı dosyaların listelenmesi
filePath = f"./{eventname}/{eventlist[0]}"
filePath2 = f"./{eventname}/{eventlist[1]}"
time,acceleration,npts,dt = rmd.ReadRecord(filePath,g=1,plot=1)
time2,acceleration2,npts2,dt2 = rmd.ReadRecord(filePath2,g=1,plot=1)

#timegap = np.arange(time[-1],time[-1]+100+dt,dt)
#accgap  = np.arange(0,len(timegap))*0
#
#time = np.append(time,timegap)
#acceleration = np.append(acceleration,accgap)
#plt.plot(time,acceleration)

In [ ]:
GMFile ="C:\\Users\\Lenovo\\Desktop\\part1\\Mammoth_Lakes\\eqe.acc"

GMdirection=1
GMfact=1
tsTag=3
IDloadTag=400
GMfatt = Unit.g*GMfact
#ops.timeSeries('Path', tsTag, '-dt', dt, '-values', *acceleration, '-factor', GMfatt) # time series object
#ops.pattern('UniformExcitation', IDloadTag,GMdirection, '-accel', tsTag)# pattern object
ops.timeSeries('Path', tsTag, '-dt', dt, '-filePath', GMFile, '-factor', GMfatt) # time series object
ops.pattern('UniformExcitation', IDloadTag,GMdirection, '-accel', tsTag)# pattern object



ops.wipeAnalysis()

bd.analysis_define()

top_nodal_disp,eleForces,time,sectionStrainStress,Eds,eleCurvature =bd.run_timehistory(DtAnalysis=dt,TmaxAnalysis=20)
bd.top_disp_plot(top_nodal_disp,time,dt)
T1, Mratios, Mfactors, Mtots = bd.modal_analys2(3)
ops.loadConst('-time', 0.0)


In [ ]:
eleCurvature[1][5]

In [ ]:
GMFile ="C:\\Users\\Lenovo\\Desktop\\part1\\Mammoth_Lakes\\eqe2.acc"

GMdirection=1
GMfact=1.0
tsTag=4
IDloadTag=401
GMfatt = Unit.g*GMfact
#ops.timeSeries('Path', tsTag, '-dt', dt, '-values', *acceleration, '-factor', GMfatt) # time series object
#ops.pattern('UniformExcitation', IDloadTag,GMdirection, '-accel', tsTag)# pattern object
ops.timeSeries('Path', tsTag, '-dt', dt, '-filePath', GMFile, '-factor', GMfatt) # time series object
ops.pattern('UniformExcitation', IDloadTag,GMdirection, '-accel', tsTag)# pattern object



ops.wipeAnalysis()

analysis_define()

top_nodal_disp2,time2,Eds2=run_analysis(DtAnalysis=dt,TmaxAnalysis=time[-1])
bd.top_disp_plot(top_nodal_disp2,time2,dt)
T2, Mratios, Mfactors, Mtots = bd.modal_analys2(3)
ops.loadConst('-time', 0.0)

In [ ]:
GMFile ="C:\\Users\\Lenovo\\Desktop\\part1\\Mammoth_Lakes\\eqe3.acc"

GMdirection=1
GMfact=1
tsTag=5
IDloadTag=402
GMfatt = Unit.g*GMfact
#ops.timeSeries('Path', tsTag, '-dt', dt, '-values', *acceleration, '-factor', GMfatt) # time series object
#ops.pattern('UniformExcitation', IDloadTag,GMdirection, '-accel', tsTag)# pattern object
ops.timeSeries('Path', tsTag, '-dt', dt, '-filePath', GMFile, '-factor', GMfatt) # time series object
ops.pattern('UniformExcitation', IDloadTag,GMdirection, '-accel', tsTag)# pattern object



ops.wipeAnalysis()

analysis_define()

top_nodal_disp3,time3,Eds3 =run_analysis(DtAnalysis=dt,TmaxAnalysis=time[-1])
bd.top_disp_plot(top_nodal_disp3,time3,dt)
T3, Mratios, Mfactors, Mtots = bd.modal_analys2(3)
ops.loadConst('-time', 0.0)

In [ ]:
GMFile ="C:\\Users\\Lenovo\\Desktop\\part1\\Mammoth_Lakes\\eqe4.acc"

GMdirection=1
GMfact=2.0
tsTag=6
IDloadTag=403
GMfatt = Unit.g*GMfact
#ops.timeSeries('Path', tsTag, '-dt', dt, '-values', *acceleration, '-factor', GMfatt) # time series object
#ops.pattern('UniformExcitation', IDloadTag,GMdirection, '-accel', tsTag)# pattern object
ops.timeSeries('Path', tsTag, '-dt', dt, '-filePath', GMFile, '-factor', GMfatt) # time series object
ops.pattern('UniformExcitation', IDloadTag,GMdirection, '-accel', tsTag)# pattern object



ops.wipeAnalysis()

analysis_define()

top_nodal_disp4,time4,Eds4 =run_analysis(DtAnalysis=dt,TmaxAnalysis=time[-1])
bd.top_disp_plot(top_nodal_disp4,time4,dt)
T4, Mratios, Mfactors, Mtots = bd.modal_analys2(3)
ops.loadConst('-time', 0.0)

In [ ]:
GMFile ="C:\\Users\\Lenovo\\Desktop\\part1\\Mammoth_Lakes\\eqe5.acc"

GMdirection=1
GMfact=1.0
tsTag=7
IDloadTag=404
GMfatt = Unit.g*GMfact
#ops.timeSeries('Path', tsTag, '-dt', dt, '-values', *acceleration, '-factor', GMfatt) # time series object
#ops.pattern('UniformExcitation', IDloadTag,GMdirection, '-accel', tsTag)# pattern object
ops.timeSeries('Path', tsTag, '-dt', dt, '-filePath', GMFile, '-factor', GMfatt) # time series object
ops.pattern('UniformExcitation', IDloadTag,GMdirection, '-accel', tsTag)# pattern object



ops.wipeAnalysis()

analysis_define()

top_nodal_disp5,time5,Eds5 =run_analysis(DtAnalysis=dt,TmaxAnalysis=time[-1])
bd.top_disp_plot(top_nodal_disp5,time5,dt)
T5, Mratios, Mfactors, Mtots = bd.modal_analys2(3)
ops.loadConst('-time', 0.0)

In [ ]:
concat_disp =[i for i in top_nodal_disp[16]]
for i in top_nodal_disp2[16]:
    concat_disp.append(i)
for i in top_nodal_disp3[16]:
    concat_disp.append(i)
for i in top_nodal_disp4[16]:
    concat_disp.append(i)
for i in top_nodal_disp5[16]:
    concat_disp.append(i)
concat_time = [i*dt for i in range(len(concat_disp))]

%matplotlib inline
plt.rcParams.update({'font.size': 16})
fig, ax = plt.subplots(figsize=(25,10))
fig.subplots_adjust(bottom=0.15, left=0.2)
ax.grid()
ax.plot(concat_time,concat_disp,linewidth=2)
ax.axhline(0, color='black', lw=2)
ax.set(xlabel="Time [Sec]", ylabel=f"Horizontal Displacement of node {max(ops.getNodeTags())}",title=f"{eventname} main-after shock top disp/time")
plt.show()

|No|Periods|
|---|------|
|elastic  |0.8456|
|1.EQE |0.9292|
|2.EQE  |0.9289|
|3.EQE  | 0.9294|
|4.EQE  |0.9642|
|5.EQE  | 0.9639|


In [ ]:
def calc_timegap(Td,damp_ratio,T):
    """
    Td          : Kuvvetli yer hareketi süresi
    damp_ratio  : Yapının sönüm oranı
    T           : Yapının doğal titreşim periyodu

    """
    R_rest = Td*(0.05/damp_ratio)*((21.8559*T+0.0258)*(Td**(-0.9982))+0.0214)
    R_timegap = round(R_rest,0)
    return R_timegap

In [ ]:
calc_timegap(50,0.05,0.6)